In [1]:
pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.generativeai as genai

In [3]:
#API key for Gemini
api_key = "ENTER_KEY"
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash-lite")

In [4]:
prompt = "I'm a girl and I'm riding the subway, coming back late after a party. I am in Paris. It seems to me that a man is stalking me and I am afraid of him. Give me instructions on what to do?"

In [5]:
response = model.generate_content(
    contents=[{"role": "user", "parts": [{"text": prompt}]}],
    generation_config={
        "temperature": 0,  
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 4096,
        "response_mime_type": "text/plain"
    }
)

print(response.text)


InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

In [ ]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from IPython.display import Audio
from IPython.display import Video
import torch
import librosa
import os
import pandas as pd

### Dataset with audio

In [ ]:
data=pd.read_csv('/kaggle/input/911-recordings/911_recordings/911_metadata.csv')
print(data.columns.tolist())
print(data['deaths'].value_counts())

In [ ]:
data.head(10)

In [ ]:
data.set_index('id', inplace=True)

print(data.loc[1])

In [ ]:
!pip install transformers torchaudio librosa --quiet


In [ ]:
data = pd.read_csv('/kaggle/input/911-recordings/911_recordings/911_metadata.csv')

row = data[data['id'] == 1].iloc[0]
audio_filename = row['file_name']
audio_path = os.path.join('/kaggle/input/911-recordings/911_recordings', audio_filename)


In [ ]:
input_audio, _ = librosa.load(audio_path, sr=16000)

#Transcribe with Wav2Vec2
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

input_values = tokenizer(input_audio, return_tensors="pt", padding="longest").input_values

with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]

print("Transcription:\n", transcription)

In [ ]:
# Construct prompt
prompt = f"""This is a transcription of a 911 call:
\"\"\"
{transcription}
\"\"\"

What do you understand about the emergency? Can you summarize what's happening and identify if any action needs to be taken?
"""

# Generate response
response = model.generate_content(
    contents=[{"role": "user", "parts": [{"text": prompt}]}],
    generation_config={
        "temperature": 0,
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 4096,
        "response_mime_type": "text/plain"
    }
)

# Show the model's reply
print("Gemini's response:\n", response.text)

#### Only using description

In [ ]:
data = pd.read_csv('/kaggle/input/911-recordings/911_recordings/911_metadata.csv')
print(data.columns.tolist())
print(data[['id', 'title', 'description']].head(10))


In [ ]:
row = data[data['id'] == 128].iloc[0]

row['description']

In [ ]:
prompt = f"""
This is a 911 call description. Please summarize the event and advise how to help:

{row['description']}
"""

response = model.generate_content(
    contents=[{"role": "user", "parts": [{"text": prompt}]}],
    generation_config={
        "temperature": 0,
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 4096,
        "response_mime_type": "text/plain"
    }
)

print(response.text)


### Another dataset with wide descriptions

In [ ]:
file_path = '/kaggle/input/montcoalert/911.csv' 
df = pd.read_csv(file_path)

df.head(10)

In [ ]:
def request_assistance_with_llm(row_index):
    row = df.iloc[row_index]
    
    prompt = f"""
    You are an emergency assistant. Your job is to give calm, clear, step-by-step instructions 
    to someone in an emergency situation. You are not a chatbot. You never ask questions. 
    You respond with practical advice, immediately, based only on the information given.

    If the user mentions danger to life, safety, or health, prioritize those in your instructions.
    If possible, suggest calling emergency services or seeking nearby help, but always stay specific.

    Situation:
    {row['desc']}
    
    Instructions:
    """
    
    response = model.generate_content(
        contents=[{"role": "user", "parts": [{"text": prompt}]}],
        generation_config={
            "temperature": 0,
            "top_p": 0.95,
            "top_k": 40,
            "max_output_tokens": 4096,
            "response_mime_type": "text/plain"
        }
    )
    
    print(f"Response for case at {row['addr']}, {row['twp']}:\n{response.text}\n")



In [ ]:
request_assistance_with_llm(1)

#### After it can be a dialogue

### Lets try falcon-7b-instruct

In [ ]:
from transformers import pipeline

df = pd.read_csv("/kaggle/input/montcoalert/911.csv")
pipe = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", trust_remote_code=True)

row_index = 1
row = df.iloc[row_index]

In [ ]:
prompt = (
    "You are an emergency assistant. Your job is to give calm, clear, step-by-step instructions "
    "to someone in an emergency situation. You are not a chatbot. You never ask questions. "
    "You respond with practical advice, immediately, based only on the information given.\n\n"
    "If the user mentions danger to life, safety, or health, prioritize those in your instructions.\n"
    "If possible, suggest calling emergency services or seeking nearby help, but always stay specific.\n\n"
    f"Situation:\n{row['desc']}\n\n"
    "Instructions:"
)

# Generate response
response = pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0)

# Print result
print(f"Prompt:\n{prompt}\n")
print(f"Response:\n{response[0]['generated_text'][len(prompt):]}")
